# **01_data_parsing.ipynb**

В этом ноутбуке собираются данные о новостройках Москвы.

Данные получены с помощью парсинга с сайта Move.ru.

Результат ноутбука — таблица с сырыми объектами (цены, площади, адреса, девелопер и др.), которая используется в дальнейшей обработке и анализе.

In [ ]:
pip install requests beautifulsoup4 fake-useragent lxml

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 94.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.0/512.0 kB 37.6 MB/s eta 0:00:00


In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import time
import random
import logging
from fake_useragent import UserAgent

Сначала парсим главную страницу с предложенными объявлениями и смотрим где находятся ссылки, которые нам надо вычленить

In [ ]:
url = 'https://move.ru/kvartiry_v_novostroykah/v_predelah_mkad/'
page = requests.get(url)
soup = BeautifulSoup(page.text, 'html.parser')
print(soup.prettify())

вычленяем эти самые ссылки с 400 страниц (в итоге вышло меньше, поскольку я по несколько раз продолжала парсить). На каждой странице по 30 ссылок

In [ ]:
# Создаем экземпляр UserAgent для генерации случайных заголовков браузера
ua = UserAgent()

def get_page_with_retry(url, max_retries=3):

    # Цикл по количеству попыток
    for attempt in range(max_retries):
        try:
            # Формируем заголовки HTTP-запроса
            headers = {
                'User-Agent': ua.random, # Случайный User-Agent
                'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8', # Типы контента, которые принимаем
                'Accept-Language': 'ru-RU,ru;q=0.8,en-US;q=0.5,en;q=0.3', # Предпочтительные языки
                'Accept-Encoding': 'gzip, deflate, br', # Поддерживаемые методы сжатия
                'Connection': 'keep-alive', # Поддержание соединения
                'Upgrade-Insecure-Requests': '1', # Запрос на обновление небезопасных запросов
            }
            # Выполняем GET-запрос с заданными параметрами
            response = requests.get(
                url,
                headers=headers,
                timeout=10,
                allow_redirects=True # Разрешаем перенаправления
            )

            if response.status_code == 200:
                return response
            else:
                print(f"Попытка {attempt + 1}: Статус код {response.status_code} для {url}")

        except requests.exceptions.RequestException as e:
            print(f"Попытка {attempt + 1}: Ошибка при запросе {url}: {e}")

        if attempt < max_retries - 1:
            time.sleep(random.uniform(2, 5))

    return None

def extract_urls_from_page(soup, base_url):
    urls = []
    for link in soup.find_all('a', href=True):
        href = link['href']
        if re.search(r'/objects/[\w-]+\s*[\w\s]*\d+', href):
            if href not in urls:
                urls.append(href)
    return urls

def main():
    base_url = 'https://move.ru/kvartiry_v_novostroykah/v_predelah_mkad/'
    all_urls = []
    max_pages = 400

    for page_num in range(1, max_pages + 1):
        print(f"Обрабатывается страница {page_num} из {max_pages}...")

        # Формируем URL страницы (для первой страницы без параметра page)
        if page_num == 1:
            page_url = base_url
        else:
            page_url = f"{base_url}?page={page_num}"

        # Получаем страницу
        response = get_page_with_retry(page_url)

        if response is None:
            print(f"Не удалось получить страницу {page_num}, пропускаем...")
            continue

        # Парсим HTML
        soup = BeautifulSoup(response.text, 'html.parser')

        # Извлекаем URL
        page_urls = extract_urls_from_page(soup, base_url)
        all_urls.extend(page_urls)

        time.sleep(random.uniform(1, 3))

    all_urls = list(set(all_urls))

    with open('extracted_urls.txt', 'w', encoding='utf-8') as f:
        for url in all_urls:
            f.write(url + '\n')

    print(f"Результаты сохранены в файл 'extracted_urls.txt'")

if name == "main":
    main()

Открываем сохраненный файл с ссылками

In [ ]:
with open('extracted_urls.txt', 'r', encoding='utf-8') as file:
    urls = [line.strip() for line in file if line.strip()]

Найдено 10496 ссылок для парсинга


Теперь парсим одно из объявлений и смотрим его код, где находятся признаки, которые мы хотимм

In [ ]:
url = 'https://move.ru/objects/prodaetsya_2-komnatnaya_kvartira_ploschadyu_604_kvm_moskva_mojayskiy_rayon_ulica_vereyskaya_d_29s35_9285880157/'
page = requests.get(url)
soup = BeautifulSoup(page.text, 'html.parser')
print(soup.prettify())

In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import time
import random

# Создаем сессию для сохранения cookies и заголовков между запросами
session = requests.Session()
# Устанавливаем стандартные заголовки для имитации браузера
session.headers.update({
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
})

# Предкомпилируем регулярные выражения
regex_patterns = {
    'id': re.compile(r'_(\d+)/?$'),  # _ затем одна/несколько цифр, затем необязательный / и конец строки $
    'price': re.compile(r'[\d\s\xa0]+'), # один/несколько символов из класса: цифры, пробелы, неразрывные пробелы
    'year': re.compile(r'(\d{4})'), #точно 4 цифры подряд
    'rooms': re.compile(r'(\d+)'), # одна/несколько цифр
    'area': re.compile(r'(\d+(?:\.\d+)?)'), # цифры, затем необязательная незахватываемая группа (?:) с точкой и цифрами
    'floor_current': re.compile(r'(\d+)/\d+'), # цифры, затем /, затем цифры
    'floor_total': re.compile(r'/(\d+)'), # символ /, затем цифры
    'views': re.compile(r'(\d+)'), # одна/несколько цифр
    'distance_km': re.compile(r'([\d,]+)\s*км'), # цифры/запятые, затем пробелы, затем текст км
    'distance_m': re.compile(r'(\d+)\s*м') # цифры, затем пробелы, затем текст м
}

all_data = []

for i, url in enumerate(urls, 1):
    try:
        if i%5 == 0:
            print(f"Парсим {i}/{len(urls)}")

        # Выполняем GET-запрос к странице объявления
        response = session.get(url, timeout=8)
        # Парсим HTML-контент с помощью BeautifulSoup и lxml парсера
        soup = BeautifulSoup(response.content, 'lxml')

        data = {}

        # 1. Айди
        id_match = regex_patterns['id'].search(url)
        data['id'] = id_match.group(1) if id_match else None

        # 2. Цена в миллионах
        price_elem = soup.find('span', class_='card-objects-price__main-price')
        if price_elem:
            price_text = price_elem.get_text(strip=True)
            # Убираем все нецифровые символы кроме точек и запятых
            clean_price = re.sub(r'[^\d,]', '', price_text.replace('\xa0', ''))
            clean_price = clean_price.replace(',', '.')
            if clean_price:
                try:
                    price = float(clean_price)
                    data['price_millions'] = round(price / 1000000, 2)
                except ValueError:
                    print(f"Не удалось преобразовать цену: {price_text}")

        # 3-5. Название ЖК + застройщик + класс жилья
        spec_items = soup.find_all('div', class_='card-specifications-table__item')
        count = 0
        for item in spec_items:
            desc_elem = item.find('span', class_='card-specifications-table__description')
            value_elem = item.find('span', class_='card-specifications-table__title')
            link_elem = item.find('a', class_='card-specifications-table__link')

            # если мы нашли все три признака заканчиваем и не обрабатываем дальше
            if count == 3:
                  break

            if desc_elem:
                desc_text = desc_elem.get_text(strip=True)

                # Название ЖК
                if desc_text == 'Название ЖК':
                  data['complex_name'] = link_elem.get_text(strip=True)
                  count += 1

                # Застройщик
                elif desc_text == 'Застройщик':
                    data['developer'] = link_elem.get_text(strip=True)
                    count += 1

                # Класс жилья
                elif desc_text == 'Класс жилья':
                    data['housing_class'] = value_elem.get_text(strip=True)
                    count += 1


        # 6-12. Основные данные (площади, этажи, комнаты)
        table_items = soup.find_all('div', class_='card-specifications-table__item')

        count = 0

        for item in table_items:
            desc_elem = item.find('span', class_='card-specifications-table__description')
            title_elem = item.find('span', class_='card-specifications-table__title')

             # если мы нашли все пять признаков, заканчиваем и не обрабатываем дальше
            if count == 5:
              break

            if desc_elem and title_elem:
                desc = desc_elem.get_text(strip=True)
                title = title_elem.get_text(strip=True)

                if 'Общая площадь' in desc and 'total_area' not in data:
                    match = regex_patterns['area'].search(title)
                    if match:
                        data['total_area'] = float(match.group(1))
                        count += 1
                elif 'Жилая площадь' in desc:
                    match = regex_patterns['area'].search(title)
                    if match:
                        data['living_area'] = float(match.group(1))
                        count += 1
                elif 'Площадь кухни' in desc:
                    match = regex_patterns['area'].search(title)
                    if match:
                        data['kitchen_area'] = float(match.group(1))
                        count += 1
                elif 'Этаж' in desc and 'floor' not in data:
                    match = regex_patterns['floor_current'].search(title)
                    if match:
                        data['floor'] = int(match.group(1))
                    match = regex_patterns['floor_total'].search(title)
                    if match:
                        data['total_floors'] = int(match.group(1))
                    count += 1
                elif 'Количество комнат' in desc and 'rooms' not in data:
                    match = regex_patterns['rooms'].search(title)
                    if match:
                        data['rooms'] = int(match.group(1))
                        count += 1

        # 13. Полный адрес
        address_link = soup.find('a', class_='base-link card-objects-location__address-link')
        data['full_address'] = address_link['title'] if address_link and address_link.get('title') else None

        # 14-17. Метро
        metro_data = []
        metro_stations = soup.find_all('li', class_='card-objects-near-stations__station')

        for station in metro_stations:
            name_elem = station.find('a', class_='card-objects-near-stations__station-link')
            duration_elem = station.find('span', class_='card-objects-near-stations__station-duration')
            distance_elem = station.find('span', class_='card-objects-near-stations__station-distance')

            if name_elem:
                metro_name = name_elem.get_text(strip=True)

                # Парсим расстояние
                distance_m = None
                if distance_elem:
                    distance_text = distance_elem.get_text(strip=True)
                    km_match = regex_patterns['distance_km'].search(distance_text)
                    if km_match:
                        distance_m = int(float(km_match.group(1).replace(',', '.')) * 1000) # group(1) это то, что было захвачено в первых скобках ([\d,]+) регулярного выражения
                    else:
                        m_match = regex_patterns['distance_m'].search(distance_text)
                        if m_match:
                            distance_m = int(m_match.group(1))

                metro_data.append({
                    'name': metro_name,
                    'distance_m': distance_m
                })

        # Сортируем по расстоянию и берем ближайшее
        if metro_data:
            metro_data.sort(key=lambda x: x['distance_m'] if x['distance_m'] else float('inf'))
            nearest = metro_data[0]

            data['metro_data'] = metro_data
            data['metro_names'] = [m['name'] for m in metro_data]
            data['nearest_metro'] = nearest['name']
            data['nearest_metro_distance'] = nearest['distance_m']

        # 18. Просмотры
        views_elems = soup.find_all('span', class_='card-meta__item')
        for elem in views_elems:
            views_text = elem.get_text(strip=True)
            if 'просмотр' in views_text.lower():
                views_match = regex_patterns['views'].search(views_text)
                if views_match:
                    data['views'] = int(views_match.group(1))
                    break

        data['url'] = url
        all_data.append(data)
        time.sleep(random.uniform(0.1, 0.5))
        if i % 100 == 0:
            time.sleep(random.uniform(1.5, 2))

    except Exception as e:
        print(f"Ошибка при парсинге {url}: {e}")
        continue

# Создаем DataFrame
df = pd.DataFrame(all_data)

print(f"\nУспешно собрано {len(df)} объявлений")

# Сохраняем в CSV
df.to_csv('parsingMOVE.csv', index=False, encoding='utf-8')
print("\nДанные сохранены в parsingMOVE.csv")

Найдено 10496 ссылок для парсинга
Парсим 5/1000
Парсим 10/1000
Парсим 15/1000
Парсим 20/1000
Парсим 25/1000
Парсим 30/1000
Парсим 35/1000
Парсим 40/1000
Парсим 45/1000
Парсим 50/1000
Парсим 55/1000
Парсим 60/1000
Парсим 65/1000
Парсим 70/1000
Парсим 75/1000
Парсим 80/1000
Парсим 85/1000
Парсим 90/1000
Парсим 95/1000
Парсим 100/1000
Парсим 105/1000
Парсим 110/1000
Парсим 115/1000
Парсим 120/1000
Парсим 125/1000
Парсим 130/1000
Парсим 135/1000
Парсим 140/1000
Парсим 145/1000
Парсим 150/1000
Парсим 155/1000
Парсим 160/1000
Парсим 165/1000
Парсим 170/1000
Парсим 175/1000
Парсим 180/1000
Парсим 185/1000
Парсим 190/1000
Парсим 195/1000
Парсим 200/1000
Парсим 205/1000
Парсим 210/1000
Парсим 215/1000
Парсим 220/1000
Парсим 225/1000
Парсим 230/1000
Парсим 235/1000
Парсим 240/1000
Парсим 245/1000
Парсим 250/1000
Парсим 255/1000
Парсим 260/1000
Парсим 265/1000
Парсим 270/1000
Парсим 275/1000
Парсим 280/1000
Парсим 285/1000
Парсим 290/1000
Парсим 295/1000
Парсим 300/1000
Парсим 305/1000
Парсим 310